# STAC-SI(-CRED)
#### SpatioTemporal Asset Catalog for Space Intelligence

Create a STAC of Space Intelligence data. The data is stored in a public GCP bucket `gs://stac-si`, for private data we use `gs://stac-si-cred`. 

We'll translate existing metadata about each dataset to STAC information, utilizing the `eo`, `view`, and `proj` extensions. Finally we'll write the STAC catalog to json, allowing us to use [stac-browser](https://github.com/radiantearth/stac-browser) to preview the images.

#### Functions
- Upload tif to gcloud bucket
- Auto translate tif to COG before upload to gcloud
- Create stac collection
- Index `gs` data into collection
- Validate and save collection
- Host collection.json in public repo

#### WiP / TODO:
- seamless private data access ~ CORS configuration
- multi-band ingestion flow (item can hold multiple bands, stored as separate assets)

Credits to [sat-stac-landsat](https://github.com/sat-utils/sat-stac-landsat) and [pystac tutorial](https://github.com/stac-utils/pystac/blob/main/docs/tutorials/creating-a-landsat-stac.ipynb).

In [1]:
# imports
from datetime import datetime
import glob
import os
import pystac    
from pystac.extensions.eo import Band
import rasterio
import rio_stac
from osgeo import gdal
from dateutil.parser import parse
from google.cloud import storage
from localsecret import gs_access_key, gs_secret_access_key, stacsi_repo_path, service_account_json, data_path

from pathlib import Path
import os
from http.server import HTTPServer, SimpleHTTPRequestHandler


In [2]:
# Set up gcloud authentication:
# https://cloud.google.com/storage/docs/reference/libraries#setting_up_authentication
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/path/to/file.json"  
# https://stackoverflow.com/a/46651026

service_account_json_path = f"{stacsi_repo_path}{service_account_json}"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_json_path
os.environ['GS_SECRET_ACCESS_KEY'] = gs_secret_access_key
os.environ['GS_ACCESS_KEY_ID'] = gs_access_key

# Check gcloud connection
def implicit():
    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()
    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)
    

implicit()

[<Bucket: alospalsar>, <Bucket: samples_ai4g>, <Bucket: stac-si>, <Bucket: stac-si-cred>]


In [3]:
# Function to upload data to bucket
# https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python
# https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/storage/cloud-client/storage_upload_file.py#L23

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"  # The ID of your GCS bucket
    # source_file_name = "local/path/to/file"  # The path to your file to upload
    # destination_blob_name = "storage-object-name"   # The ID of your GCS object
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")


def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"
    storage_client = storage.Client()
    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)
    blob_names = []
    for blob in blobs:
        blob_names.append(blob.name)
    return blob_names


def upload_tif_to_bucket(tif_fp, bucket_name, existing_blobs=[], create_thumbnail=True):
    """add the tif and optionally a jpeg thumbnail of it to a gs bucket"""
    # skip folders in bucket storage (check if this grows to prevent duplicates)
    p, f = os.path.split(tif_fp)
    fname, ext = os.path.splitext(f)
    tif_blob = f"{fname}{ext.upper()}"  # upper 'tif' to 'TIF' (don't know why, but works)
    # add that data to the bucket
    if tif_blob not in existing_blobs:
        print(f"Uploading {tif_fp} to bucket and store as {tif_blob}")
        upload_blob(bucket_name, tif_fp, tif_blob)
        
    if create_thumbnail:  # Who doesn't like a preview? Convert data to thumbnail thumb
        thumb_fp = f"{os.path.splitext(tif_fp)[0]}.JPEG"
        from PIL import Image
        with rasterio.open(tif_fp) as src:
            # Retrieve the smallest thumbnail
            oview = src.overviews(1)[-1]
            thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
            im = Image.fromarray(thumbnail)
            if im.mode != 'RGB':
                im = im.convert('RGB')
            im.save(thumb_fp)
        _, thumb_blob = os.path.split(thumb_fp)
        if thumb_blob not in existing_blobs:
            upload_blob(bucket_name, thumb_fp, thumb_blob)
    else:
        thumb_blob = None
    return tif_blob, thumb_blob


def tif_data_dt(tif_blob):
    """try: find datetime in filename, except: use date of adding the data"""
    try:
        data_dt = parse(tif_blob, fuzzy=True)
    except:  
        data_dt = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
    return data_dt


def add_assets(item, thumb_href, band_info_dict):
    """add bands and assets to item"""
    # Add bands
    for band_id, band_info in band_info_dict.items():
        band_url = f"https://storage.googleapis.com/{bucket_name}/{band_id}"
        # band_url = f"gs://stac-si/{band_id}"  # + band_id
        asset = pystac.Asset(href=band_url, media_type=pystac.MediaType.COG)
        bands = [band_info['band']]
        item.ext.eo.set_bands(bands, asset)
        item.add_asset(band_id, asset)
        
        # If this asset has a different GSD than the item, set it on the asset
        if band_info['gsd'] != item.common_metadata.gsd:
            item.common_metadata.set_gsd(band_info['gsd'], asset)
        
    # Add other assets
    item.add_asset("thumbnail", pystac.Asset(href=thumb_href, media_type=pystac.MediaType.JPEG))
    return item


def raster_info(tif):
    """get ground sampling distance gsd in meters (e.g. 20) and epsg nr (e.g. 27700)"""
    raster = rasterio.open(tif)
    pixelSizeX, pixelSizeY  = raster.res
    gsd = round((abs(pixelSizeX) + abs(pixelSizeY)) / 2, 1)
    if raster.crs.is_epsg_code:
        epsg_nr = int(raster.crs['init'].lstrip('epsg:'))
    else:
        epsg_nr = None
    #     if 'UTM_ZONE' in metadata:
    #         center_lat = (min_lat + max_lat)/2.0
    #         epsg_nr = int(('326' if center_lat > 0 else '327') + metadata['UTM_ZONE'])
    return gsd, epsg_nr

    
def create_stac_si(bucket_name):
    """Create collection template"""
    collection_id = f"{bucket_name}"
    collection_title = 'Private STAC of Space Intelligence'
    collection_title = 'Public STAC of Space Intelligence'
    collection_description = "SpatioTemporal Asset Catalog is the collection of Space Intelligence assets, crawlable through standardized STAC APIs."
    spatial_extent = pystac.SpatialExtent([[-180, -90, 180, 90]])
    temporal_extent = pystac.TemporalExtent([[datetime(1970, 1, 1), None]])  # Unix epoch is 00:00:00 UTC on 1 January 1970
    collection_extent = pystac.Extent(spatial_extent, temporal_extent)
    collection = pystac.Collection(id=collection_id,
                         title=collection_title,
                         description=collection_description, 
                         extent=collection_extent)

    # collection_license = 'PDDL-1.0'
    collection.providers = [
        pystac.Provider(name='USGS', roles=['producer'], url='https://landsat.usgs.gov/'),
        pystac.Provider(name='ESA', roles=['producer'], url='https://sentinels.copernicus.eu/web/sentinel/home'),
        pystac.Provider(name='NASA', roles=['producer'], url='https://gedi.umd.edu/'),
        pystac.Provider(name='JAXA', roles=['producer'], url='https://www.eorc.jaxa.jp/ALOS/en/about/palsar.htm'),
        pystac.Provider(name='GCP', roles=['host'], url='https://gee.stac.cloud/'),
        pystac.Provider(name='AWS', roles=['host'], url='https://registry.opendata.aws/sentinel-2-l2a-cogs/'),
    ]
    return collection    
    

def tif2collection(tif, collection, bucket_name, existing_blobs):
    """Add data to collection by looping through the tifs"""
    # If tif not in bucket, upload the tif to the bucket. 
    # TODO overwrite = True options (e.g. if new version)
    tif_blob, thumb_blob = upload_tif_to_bucket(tif, bucket_name, existing_blobs, create_thumbnail=True)
    # Add item to the local stac as stac item (data in bucket are called blobs)
    data_dt = tif_data_dt(tif_blob)  # assign a datetime to the tif
    tif_blob_path = f"https://storage.googleapis.com/{bucket_name}/{tif_blob}"
    # tif_blob_path = f"{cloud}{bucket_name}/{tif_blob}"  # define path to blob
    # Note that we use rio_stac alternative to pystac's method
    # item = pystac.Item(id=item_id, datetime=item_datetime, geometry=item_geometry, bbox=item_bbox, properties={})```
    print(tif_blob_path)
    item = rio_stac.stac.create_stac_item(tif_blob_path, input_datetime=data_dt)  # create stac item
    # Stac items contain assets, which are the files, these can have separate metadata that we add below
    item.ext.enable('eo')
    name, ext = os.path.splitext(tif_blob)
    gsd, epsg = raster_info(tif)
    band_info_dict = {tif_blob: {'band': Band.create(name=name, common_name=name),'gsd': gsd},}
    # thumb_href = f"gs://stac-si/{thumb_blob}"
    thumb_href = f"https://storage.googleapis.com/{bucket_name}/{thumb_blob}"
    item = add_assets(item, thumb_href, band_info_dict)
    item.ext.enable('projection') 
    item.ext.projection.epsg = epsg
    item.common_metadata.gsd = gsd
    item.validate()
    collection.add_item(item)
    return collection


def validate_and_save_collection(collection, root_path):
    collection.update_extent_from_items()
    collection.normalize_hrefs(root_path)

    collection.validate_all()
    collection.save(pystac.CatalogType.SELF_CONTAINED)
    return collection


def tif2cog(tif, dtype="Byte"):
    print("converting tif to cog")
    name, ext = os.path.splitext(tif)
    tmp_tif = f"{name}_tif{ext}"
    os.rename(tif, tmp_tif)
    !gdalwarp $tmp_tif $tif -of COG -co COMPRESS=DEFLATE -ot $dtype
    print(tmp_tif)
    os.remove(tmp_tif)
    return tif


def cors_configuration(bucket_name):
    """Set a bucket's CORS policies configuration."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    bucket.cors = [
        {
            "origin": ["*"],
            "responseHeader": [
                "Content-Type",
                "x-goog-resumable"],
            "method": ['PUT', 'POST'],
            "maxAgeSeconds": 3600
        }
    ]
    bucket.patch()

    print("Set CORS policies for bucket {} is {}".format(bucket.name, bucket.cors))
    return bucket



In [4]:
# List data and define bucket
bucket_name = "stac-si-cred"  # private stac of space intelligence
bucket_name = "stac-si"  # public stac of space intelligence
tifs = glob.glob(f'{data_path}{bucket_name}/*.tif')
cloud = "gs://"  # google cloud
# TODO configure private data access
# gs://stac-si | gs://stac-si-cred
# https://storage.googleapis.com/stac-si/ | https://storage.googleapis.com/stac-si-cred/
existing_blobs = list_blobs(bucket_name)
# create stac_si collection
collection = create_stac_si(bucket_name)
for tif in tifs:
    tif_is_cog = "LAYOUT=COG" in gdal.Info(tif)
    if not tif_is_cog:
        tif = tif2cog(tif, dtype="Byte")  # TODO allow other dtypes (not specifying led to weird dual band tifs.)
    collection = tif2collection(tif, collection, bucket_name, existing_blobs)
root_path = f"{stacsi_repo_path}{bucket_name}"
os.chdir(root_path)
collection = validate_and_save_collection(collection, root_path)
collection

https://storage.googleapis.com/stac-si/LandCoverScotland_2019.TIF
https://storage.googleapis.com/stac-si/LandCoverScotland_2020.TIF
https://storage.googleapis.com/stac-si/LandCoverChangeScotland_20192020.TIF


<Collection id=stac-si>

Now you can follow the stac-browser instructions for starting a stac-browser instance and point it at http://localhost:5555/collection.json to serve out the STAC!

To quit the server, use the Kernel -> Interrupt menu option.

In [ ]:
#!export CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt
# bucket = cors_configuration(bucket_name)
# bucket

import os
from http.server import HTTPServer, SimpleHTTPRequestHandler

os.chdir(root_path)

class CORSRequestHandler(SimpleHTTPRequestHandler):
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET')
        self.send_header('Cache-Control', 'no-store, no-cache, must-revalidate')
        return super(CORSRequestHandler, self).end_headers()


with HTTPServer(('localhost', 5555), CORSRequestHandler) as httpd:
    httpd.serve_forever()

127.0.0.1 - - [23/Apr/2021 09:51:24] "GET /collection.json HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 09:51:24] code 404, message File not found
127.0.0.1 - - [23/Apr/2021 09:51:24] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Apr/2021 09:52:12] "GET /collection.json HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 09:52:12] "GET /LandCoverScotland_2019.TIF/LandCoverScotland_2019.TIF.json HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 09:52:12] "GET /LandCoverScotland_2020.TIF/LandCoverScotland_2020.TIF.json HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 09:52:12] "GET /LandCoverChangeScotland_20192020.TIF/LandCoverChangeScotland_20192020.TIF.json HTTP/1.1" 200 -
